# Build a MOE framework to predict YTS/UTS/EL of all series al alloy

In [8]:
import sys
sys.path.append(r'c:\Users\20958\Desktop\NSGA_for_Al_alloys')  # 直接指定上一级目录绝对路径

## 1. Read Data and preprocess

In [10]:
import pandas as pd
import numpy as np
from DataPreprocessing import RF_DataPreprocessing
from FeatureEngineering import RF_FeatureEngineering
'''Define dataset and process dataset'''
# Define database path
path = '../dataset.xlsx'
# Read and process database 
# dataset dictionary = {property:[feature, target property],...}
data = pd.read_excel(path)
preprocessed_data = RF_DataPreprocessing(data)
# Find common low-VI features from all three properties & drop them from data
low_VI_dropped_data, grouped_VIs = RF_FeatureEngineering(preprocessed_data)

Common low VI features: {'B', 'Ga', 'Be', 'Ni', 'Pb', 'Bi', 'V'}


In [ ]:
# 构建数据集
dataset_YTS = low_VI_dropped_data['YTS']
dataset_UTS = low_VI_dropped_data['UTS']
dataset_EL = low_VI_dropped_data['EL']
dataset_EL[0].head()

,Si,Fe,Cu,Mn,Mg,Cr,Zn,Ti,Zr,Li,...,ProductionShape_Foil,ProductionShape_Forging,ProductionShape_Ingot,ProductionShape_Plate,ProductionShape_Ring,ProductionShape_Rod,ProductionShape_Sheet,ProductionShape_Tube,ProductionShape_Wire,ProductionShape_Wrought
1,-0.373583,-0.361899,-0.887721,-0.903879,-1.026145,-0.811257,-0.499924,-0.575395,-0.614524,-0.280753,...,False,False,False,False,False,False,True,False,False,False
4,-0.373583,-0.361899,-0.887721,-0.903879,-1.026145,-0.811257,-0.499924,-0.575395,-0.614524,-0.280753,...,False,False,False,False,False,False,True,False,False,False
5,-0.373583,-0.361899,-0.887721,-0.903879,-1.026145,-0.811257,-0.511890,-0.780871,-0.216876,-0.255134,...,False,False,False,False,False,False,True,False,False,False
6,-0.373583,-0.361899,-0.887721,-0.903879,-1.026145,-0.811257,-0.499924,-0.575395,-0.614524,-0.280753,...,False,False,False,False,False,False,False,True,False,False
7,-0.373583,-0.361899,-0.887721,-0.903879,-1.026145,-0.811257,-0.499924,-0.575395,-0.614524,-0.280753,...,False,False,False,False,False,False,False,False,False,False


## 2. MOE training

In [ ]:
from moe_model import ModularMoE
# 构造配置字典
config = {
    'input_dim': 52,  # 成分+工艺+系列 one-hot
    'expert_hidden': [64, 64],
    'expert_output_dim': 32,
    'num_experts': 8,
    'tasks': ['YTS', 'UTS', 'EL']
}

model = ModularMoE(config)

x = torch.randn(16, 38)  # 16 个样本
output = model(x)

print(output['YTS'].shape)  # [16]
print(output['UTS'].shape)  # [16]
print(output['EL'].shape)   # [16]
